#Mounting GDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Cropping Image

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os

#function for cropping images
def cropImg(folder_path,output_folder):

    image_files = [f for f in os.listdir(folder_path)]
    left = 100
    top = 300
    right = 2125
    bottom = 1480
    for image in image_files:
            # Open the image file
            image_path = os.path.join(folder_path, image)
            img = Image.open(image_path)

            # Crop the image using the specified dimensions
            cropped_img = img.crop((left, top, right, bottom))

            # Save the cropped image to the output folder
            output_path = os.path.join(output_folder, f"{image}")
            cropped_img.save(output_path)

In [ ]:
folder_path1="/content/drive/MyDrive/MP_Dataset/ECG Images of Myocardial Infarction Patients (240x12=2880)"
output_folder1="/content/drive/MyDrive/MP_Dataset/CroppedMI"
cropImg(folder_path1,output_folder1)

folder_path2="/content/drive/MyDrive/MP_Dataset/ECG Images of Patient that have abnormal heartbeat (233x12=2796)"
output_folder2="/content/drive/MyDrive/MP_Dataset/CroppedAB"
cropImg(folder_path2,output_folder2)

folder_path3="/content/drive/MyDrive/MP_Dataset/ECG Images of Patient that have History of MI (172x12=2064)"
output_folder3="/content/drive/MyDrive/MP_Dataset/CroppedHMI"
cropImg(folder_path3,output_folder3)

folder_path4="/content/drive/MyDrive/MP_Dataset/Normal Person ECG Images (284x12=3408)"
output_folder4="/content/drive/MyDrive/MP_Dataset/CroppedN"
cropImg(folder_path4,output_folder4)

#Agumentation is performed

In [ ]:
import cv2
import os
import numpy as np
import random

def rotate_image(image, angle):
    rows, cols, _ = image.shape
    rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (cols, rows))
    return rotated_image

def flip_image(image, flip_code):
    flipped_image = cv2.flip(image, flip_code)
    return flipped_image

def translate_image(image, tx, ty):
    translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
    translated_image = cv2.warpAffine(image, translation_matrix, (image.shape[1], image.shape[0]))
    return translated_image

def apply_augmentation(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get all ECG image files in the input folder
    ecg_files = [f for f in os.listdir(input_folder)]
    print("Number of images = ",len(ecg_files))

    for ecg_file in ecg_files:
        # Read the ECG image
        ecg_path = os.path.join(input_folder, ecg_file)
        ecg_image = cv2.imread(ecg_path)

        # Apply data augmentation

        rotated_image = rotate_image(ecg_image, angle=1)
        rotated_image1 = rotate_image(ecg_image, angle=-1)

        translation_x, translation_y = 1, 1
        translated_image = translate_image(ecg_image, translation_x, translation_y)
        translated_image1 = translate_image(rotated_image, translation_x, translation_y)

        # Save the augmented image
        output_path = os.path.join(output_folder, f"{ecg_file}")
        cv2.imwrite(output_path, ecg_image)

        output_path = os.path.join(output_folder, f"Trans{ecg_file}")
        cv2.imwrite(output_path, translated_image)

        output_path = os.path.join(output_folder, f"Rot{ecg_file}")
        cv2.imwrite(output_path, rotated_image)

        output_path = os.path.join(output_folder, f"tran{ecg_file}")
        cv2.imwrite(output_path, translated_image1)

        output_path = os.path.join(output_folder, f"ro{ecg_file}")
        cv2.imwrite(output_path, rotated_image1)

        if(output_folder=="Dataset/AugmentedHMIPatient"):
            translated_image2 = translate_image(rotated_image1, translation_x, translation_y)
            translation_x, translation_y = 1.5, 1.5
            translated_image3 = translate_image(ecg_image, translation_x, translation_y)

            output_path = os.path.join(output_folder, f"rotT{ecg_file}")
            cv2.imwrite(output_path, translated_image2)

            output_path = os.path.join(output_folder, f"Trt{ecg_file}")
            cv2.imwrite(output_path, translated_image3)

if __name__ == "__main__":
    # Specify the input folder and output folder
    input_folder="/content/drive/MyDrive/MP_Dataset/CroppedMI"
    output_folder="/content/drive/MyDrive/MP_Dataset/AugmentedMI"
    apply_augmentation(input_folder, output_folder)

    input_folder="/content/drive/MyDrive/MP_Dataset/CroppedHMI"
    output_folder="/content/drive/MyDrive/MP_Dataset/AugmentedHMI"
    apply_augmentation(input_folder, output_folder)

    input_folder="/content/drive/MyDrive/MP_Dataset/CroppedN"
    output_folder="/content/drive/MyDrive/MP_Dataset/AugmentedN"
    apply_augmentation(input_folder, output_folder)

    input_folder="/content/drive/MyDrive/MP_Dataset/CroppedAB"
    output_folder="/content/drive/MyDrive/MP_Dataset/AugmentedAB"
    apply_augmentation(input_folder, output_folder)

Number of images =  239
Number of images =  172
Number of images =  284
Number of images =  233


#  Adding the Image into Separate Array

In [ ]:
import os
normal_path= [
    os.path.join(os.getcwd(), "/content/drive/MyDrive/MP_Dataset/AugmentedN", x)
    for x in os.listdir("/content/drive/MyDrive/MP_Dataset/AugmentedN")
]
abnormal_path= [
    os.path.join(os.getcwd(), "/content/drive/MyDrive/MP_Dataset/AugmentedAB", x)
    for x in os.listdir("/content/drive/MyDrive/MP_Dataset/AugmentedAB")
]
Mi_path= [
    os.path.join(os.getcwd(), "/content/drive/MyDrive/MP_Dataset/AugmentedMI", x)
    for x in os.listdir("/content/drive/MyDrive/MP_Dataset/AugmentedMI")
]
Hmi_path= [
    os.path.join(os.getcwd(), "/content/drive/MyDrive/MP_Dataset/AugmentedHMI", x)
    for x in os.listdir("/content/drive/MyDrive/MP_Dataset/AugmentedHMI")
]

print("ECG of normal persons = " + str(len(normal_path)))
print("ECG of abnormal persons " + str(len(abnormal_path)))
print("ECG of MI persons = " + str(len(Mi_path)))
print("ECG of HMI persons = " + str(len(Hmi_path)))

ECG of normal persons = 1200
ECG of abnormal persons 1165
ECG of MI persons = 1195
ECG of HMI persons = 1140


# Assigning Labels

In [ ]:
import numpy as np

abnormal_labels = np.array([0 for _ in range(len(abnormal_path))])
normal_labels = np.array([1 for _ in range(len(normal_path))])
mi_labels= np.array([2 for _ in range(len(Mi_path))])
hmi_labels= np.array([3 for _ in range(len(Hmi_path))])


# Images and Labels as separate arrays

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from tensorflow.keras import layers, models, optimizers, regularizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import precision_score, recall_score, f1_score
import time
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Combine image paths and labels into tuples
abnormal_data = list(zip(abnormal_path, abnormal_labels))
normal_data = list(zip(normal_path, normal_labels))
mi_data = list(zip(Mi_path, mi_labels))
hmi_data = list(zip(Hmi_path, hmi_labels))

data=[]
data.extend(abnormal_data)
data.extend(normal_data)
data.extend(mi_data)
data.extend(hmi_data)

X_paths = np.array([image_path for image_path, _ in data])
y = np.array([label for _, label in data])

# SqueezeNet Model BuiltIn Function

In [ ]:
import torch

# Load the pre-trained SqueezeNet model
model1 = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)
total_params = sum(p.numel() for p in model1.parameters())
print("Total number of parameters:", total_params)
print(model1)

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=SqueezeNet1_0_Weights.IMAGENET1K_V1`. You can also use `weights=SqueezeNet1_0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/squeezenet1_0-b66bff10.pth" to /root/.cache/torch/hub/checkpoints/squeezenet1_0-b66bff10.pth
100%|██████████| 4.78M/4.78M [00:00<00:00, 99.3MB/s]


Total number of parameters: 1248424
SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(in

# Code for SqueezeNet Model

In [ ]:
import torch
import torch.nn as nn

class Fire(nn.Module):
    def __init__(self, inplanes, squeeze_planes, expand1x1_planes, expand3x3_planes):
        super(Fire, self).__init__()
        self.inplanes = inplanes
        self.squeeze = nn.Conv2d(inplanes, squeeze_planes, kernel_size=1)
        self.squeeze_activation = nn.ReLU(inplace=True)
        self.expand1x1 = nn.Conv2d(squeeze_planes, expand1x1_planes, kernel_size=1)
        self.expand1x1_activation = nn.ReLU(inplace=True)
        self.expand3x3 = nn.Conv2d(squeeze_planes, expand3x3_planes, kernel_size=3, padding=1)
        self.expand3x3_activation = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.squeeze_activation(self.squeeze(x))
        return torch.cat([
            self.expand1x1_activation(self.expand1x1(x)),
            self.expand3x3_activation(self.expand3x3(x))
        ], 1)


class SqueezeNet(nn.Module):
    def __init__(self):
        super(SqueezeNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=7, stride=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            Fire(96, 16, 64, 64),
            Fire(128, 16, 64, 64),
            Fire(128, 32, 128, 128),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            Fire(256, 32, 128, 128),
            Fire(256, 48, 192, 192),
            Fire(384, 48, 192, 192),
            Fire(384, 64, 256, 256),
            nn.MaxPool2d(kernel_size=3, stride=2, ceil_mode=True),
            Fire(512, 64, 256, 256)
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Conv2d(512, 1000, kernel_size=1),
            nn.ReLU(inplace=True),
            nn.AdaptiveAvgPool2d((1, 1))
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x.view(x.size(0), -1)

# Instantiate the model
model = SqueezeNet()
total_params = sum(p.numel() for p in model1.parameters())
print("Total number of parameters:", total_params)
print(model)

Total number of parameters: 1248424
SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(in

# LR 0.001

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split

# Define your SqueezeNet model class here...

# Define your dataset class
class CustomDataset(Dataset):
    def __init__(self, X_paths, y, transform=None):
        self.X_paths = X_paths
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X_paths)

    def __getitem__(self, idx):
        image_path = self.X_paths[idx]
        image = Image.open(image_path).convert('RGB')
        label = self.y[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Split data into train and test
X_paths = np.array([image_path for image_path, _ in data])
y = np.array([label for _, label in data])
# Split data into train and test
# Assume you have already split the data into train and test sets
X_train_paths, X_test_paths, y_train, y_test = train_test_split(X_paths, y, test_size=0.2, random_state=42)

# Create train and test datasets
train_dataset = CustomDataset(X_train_paths, y_train, transform=transform)
test_dataset = CustomDataset(X_test_paths, y_test, transform=transform)

# Define dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

avg_accuracy = correct / total
print(f"Average Accuracy: {avg_accuracy:.4f}")

# Save the model
torch.save(model.state_dict(), 'squeezenet_model.pth')


Epoch [1/10], Loss: 1.9737
Epoch [2/10], Loss: 1.4285
Epoch [3/10], Loss: 1.4265
Epoch [4/10], Loss: 1.4181
Epoch [5/10], Loss: 1.4114
Epoch [6/10], Loss: 1.4087
Epoch [7/10], Loss: 1.4175
Epoch [8/10], Loss: 1.4146
Epoch [9/10], Loss: 1.4096
Epoch [10/10], Loss: 1.4085
Average Accuracy: 0.2383


# Prediction of class (SqueezeNet LR 0.001) Acurracy 0.24%

In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Define transforms for individual images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the saved model
model = SqueezeNet()
model.load_state_dict(torch.load('squeezenet_model.pth'))
model.eval()

# Function to predict the class of an individual image
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
    _, predicted = torch.max(output, 1)
    return predicted.item()

# Example usage:
image_path = "/content/drive/MyDrive/MP_Dataset/AugmentedHMI/Copy of PMI(10).jpg"
predicted_class = predict_image(image_path)
print("Predicted class:", predicted_class)

Predicted class: 0


In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Define transforms for individual images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the saved model
model = SqueezeNet()
model.load_state_dict(torch.load('squeezenet_model.pth'))
model.eval()

# Function to predict the class of an individual image
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
    _, predicted = torch.max(output, 1)
    return predicted.item()

# Example usage:
image_path = "/content/drive/MyDrive/MP_Dataset/AugmentedN/Normal(101).jpg"
predicted_class = predict_image(image_path)
print("Predicted class:", predicted_class)

Predicted class: 0


In [ ]:
import torch
from torchvision import transforms
from PIL import Image

# Define transforms for individual images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load the saved model
model = SqueezeNet()
model.load_state_dict(torch.load('squeezenet_model.pth'))
model.eval()

# Function to predict the class of an individual image
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    image = image.unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
    _, predicted = torch.max(output, 1)
    return predicted.item()

# Example usage:
image_path = "/content/drive/MyDrive/MP_Dataset/AugmentedMI/MI(100).jpg"
predicted_class = predict_image(image_path)
print("Predicted class:", predicted_class)

Predicted class: 0


# SqueezeNet 1_0 (LR 0.0001)
Fold and accuracy

In [ ]:
model10 = torch.hub.load('pytorch/vision:v0.10.0', 'squeezenet1_0', pretrained=True)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
import numpy as np
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import KFold
from torchvision.datasets.folder import default_loader

accuracies = []
best_accuracy = 0.0
best_model = None

# Step 1: Define necessary transformations for your data
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

X_paths = np.array([image_path for image_path, _ in data])
y = np.array([label for _, label in data])


# Step 3: Define K-fold cross-validation
kf = KFold(n_splits=5, shuffle=True)

# Step 4: Iterate through the folds
for fold, (train_indices, test_indices) in enumerate(kf.split(X_paths)):
    X_train_paths, X_test_paths = X_paths[train_indices], X_paths[test_indices]
    y_train, y_test = y[train_indices], y[test_indices]

    # Step 5: Define custom dataset and data loaders
    class CustomDataset(torch.utils.data.Dataset):
        def __init__(self, X_paths, y, transform=None):
            self.X_paths = X_paths
            self.y = y
            self.transform = transform

        def __len__(self):
            return len(self.X_paths)

        def __getitem__(self, idx):
            img_path = self.X_paths[idx]
            image = default_loader(img_path)
            label = self.y[idx]
            if self.transform:
                image = self.transform(image)
            return image, label

    train_dataset = CustomDataset(X_train_paths, y_train, transform=transform)
    test_dataset = CustomDataset(X_test_paths, y_test, transform=transform)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Step 6: Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model10.parameters(), lr=0.0001)

    # Step 7: Train the model
    for epoch in range(5):  # Example: 5 epochs
        model10.train()  # Set model to training mode
        running_loss = 0.0

        for i, (inputs, labels) in enumerate(train_loader, 0):
            optimizer.zero_grad()
            outputs = model10(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader)}")

    # Step 8: Evaluate the model
    model10.eval()  # Set model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for data in test_loader:
            inputs, labels = data
            outputs = model10(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100*(correct / total)
    accuracies.append(accuracy)

    print(f"Accuracy for fold {fold+1}: {accuracy}%")

    # Step 9: Save the best model
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_model = model10

# Step 10: Calculate and print average accuracy
avg_accuracy = np.mean(accuracies)
print(f"Average accuracy: {avg_accuracy}%")

flatten_train_features = []
flatten_test_features = []
for inputs, _ in train_loader:
    with torch.no_grad():
        outputs = best_model(inputs)
    flatten_train_features.extend(outputs.cpu().numpy())
for inputs, _ in test_loader:
    with torch.no_grad():
        outputs = best_model(inputs)
    flatten_test_features.extend(outputs.cpu().numpy())

flatten_train_features = np.array(flatten_train_features)
flatten_test_features = np.array(flatten_test_features)

# Step 12: Save the best model for later use
torch.save(best_model.state_dict(), "best_model2.pth")

Epoch 1, Loss: 1.7042718948954243
Epoch 2, Loss: 0.7889223848864183
Epoch 3, Loss: 0.38302480025311647
Epoch 4, Loss: 0.22635238088856816
Epoch 5, Loss: 0.1464764740736366
Accuracy for fold 1: 97.76595744680852%
Epoch 1, Loss: 0.23204513158554496
Epoch 2, Loss: 0.06698156676701096
Epoch 3, Loss: 0.07572034698387142
Epoch 4, Loss: 0.02792254113012597
Epoch 5, Loss: 0.04788940068329655
Accuracy for fold 2: 98.08510638297873%
Epoch 1, Loss: 0.11748827453327791
Epoch 2, Loss: 0.042527111818437054
Epoch 3, Loss: 0.0033692027485017206
Epoch 4, Loss: 0.019326885311545347
Epoch 5, Loss: 0.07207109907916273
Accuracy for fold 3: 98.5956746848502%
Epoch 1, Loss: 0.05913737119063025
Epoch 2, Loss: 0.028421234210450706
Epoch 3, Loss: 0.004344418082540623
Epoch 4, Loss: 0.010415682642522534
Epoch 5, Loss: 0.004407635152709107
Accuracy for fold 4: 96.91489361702128%
Epoch 1, Loss: 0.05093581067838047
Epoch 2, Loss: 0.027570577507654574
Epoch 3, Loss: 0.008018506460618899
Epoch 4, Loss: 0.015425268471

#Classifier: Navie Bayes,Random Forest,SVM,KNN,DT

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torchvision import transforms
from torchvision.models import squeezenet1_0
from PIL import Image
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Load the SqueezeNet model
model = squeezenet1_0(pretrained=True)
model.eval()

# Define transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Define dataset paths and labels
dataset_paths = np.array([image_path for image_path, _ in data])  # Paths to your dataset images
labels = np.array([label for _, label in data])  # Corresponding labels for the dataset images

# Extract features function
def extract_features(model, image):
    features = model(image.unsqueeze(0))
    flattened_features = features.flatten().detach().numpy()
    return flattened_features

# Flatten features and collect labels
X = []
y = []

for image_path, label in zip(dataset_paths, labels):
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    flattened_features = extract_features(model.features, image)
    X.append(flattened_features)
    y.append(label)

X = np.array(X)
y = np.array(y)

# Perform 5-fold cross-validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

classifiers = {
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "KNN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB()
}

for fold, (train_index, test_index) in enumerate(skf.split(X, y)):
    print(f"Fold {fold + 1}")

    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    for classifier_name, classifier in classifiers.items():
        classifier.fit(X_train, y_train)
        y_pred = classifier.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f"{classifier_name} Accuracy: {accuracy:.4f}")

Fold 1
SVM Accuracy: 0.8012
Decision Tree Accuracy: 0.7965
Random Forest Accuracy: 0.8593
KNN Accuracy: 0.7034
Naive Bayes Accuracy: 0.6563
Fold 2
SVM Accuracy: 0.8625
Decision Tree Accuracy: 0.8136
Random Forest Accuracy: 0.8961
KNN Accuracy: 0.7392
Naive Bayes Accuracy: 0.6609
Fold 3
SVM Accuracy: 0.8961
Decision Tree Accuracy: 0.8528
Random Forest Accuracy: 0.9356
KNN Accuracy: 0.7546
Naive Bayes Accuracy: 0.6747
Fold 4
SVM Accuracy: 0.9105
Decision Tree Accuracy: 0.8785
Random Forest Accuracy: 0.9401
KNN Accuracy: 0.8098
Naive Bayes Accuracy: 0.7032
Fold 5
SVM Accuracy: 0.9436
Decision Tree Accuracy: 0.8840
Random Forest Accuracy: 0.9613
KNN Accuracy: 0.8205
Naive Bayes Accuracy: 0.7103
